In [1]:
import pandas as pd

f_Games = 'data//Steam-Games(clean).csv'
f_Items = 'data//Items(Transf).csv'
f_Reviews = 'data//Reviews(Transf).csv'

df_Games = pd.read_csv(f_Games)
df_Items = pd.read_csv(f_Items)
df_Reviews = pd.read_csv(f_Reviews)

print("User_Items")
display(df_Items.head(0))
print("Games")
display(df_Games.head(0))
print("Reviews")
display(df_Reviews.head(0))

User_Items


,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6,0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0,0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7,0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0,0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0,0
5,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,60,Ricochet,0,0
6,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,70,Half-Life,0,0
7,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,130,Half-Life: Blue Shift,0,0
8,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,300,Day of Defeat: Source,4733,0
9,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,240,Counter-Strike: Source,1853,0


Games


,id,app_name,title,publisher,genres,url,release_date,tags,reviews_url,specs,price,early_access,developer,year


Reviews


,user_id,funny,last_edited,item_id,helpful,recommend,review,posted


In [2]:
df_Games.rename(columns={'id':'item_id'}, inplace=True) #Cambio el nombre de id
df_Games = df_Games[['item_id','genres','publisher','title']]
display(df_Games.head(0))

df_Items = df_Items[['user_id','item_id','playtime_forever']]
df_Reviews = df_Reviews[['user_id','item_id','recommend']]

# Combinar User_Items, Steam_Games y Reviews en un solo DataFrame
merged_data = pd.merge(df_Items, df_Games, on="item_id", how="left")
merged_data = pd.merge(merged_data, df_Reviews, on=["user_id", "item_id"], how="left")

tam = len(merged_data)
print("Tamaño del DF: ",tam)
merged_data.head(0)

,item_id,genres,publisher,title


Tamaño del DF:  5153209


,user_id,item_id,playtime_forever,genres,publisher,title,recommend


In [3]:
tam = len(merged_data)
print("Tamaño del DF: ",tam)

merged_data.dropna(subset=['item_id'], inplace=True)
tam = len(merged_data)
print("Tamaño del DF (item_id): ",tam)
merged_data.head(0)

merged_data.dropna(subset=['genres'], inplace=True)
tam = len(merged_data)
print("Tamaño del DF: (genres)",tam)
merged_data.head(0)

merged_data.dropna(subset=['title'], inplace=True)
tam = len(merged_data)
print("Tamaño del DF: (title)",tam)
merged_data.head(0)


Tamaño del DF:  5153209
Tamaño del DF (item_id):  5153209
Tamaño del DF: (genres) 4173927
Tamaño del DF: (title) 4173927


,user_id,item_id,playtime_forever,genres,publisher,title,recommend


In [23]:
from ast import literal_eval

#Cambia la columna genres a listas
merged_data['genres'] = merged_data['genres'].apply(literal_eval)

In [24]:
# Guardar el DataFrame en un archivo CSV
total = len(merged_data)
print("Total de registros: ",total)
merged_data.to_csv('data//df_recomendacion.csv', index=False) 

Total de registros:  4173927


### Leo el archivo

In [18]:
merged_data = pd.read_csv('data//df_recomendacion.csv')
merged_data.head(0)

,user_id,item_id,playtime_forever,genres,publisher,title,recommend


In [25]:
# -----------------  Contador
import sys
def contador(ciclo, multiplo):
    if ciclo % multiplo == 0:
        sys.stdout.write("\r{}".format(ciclo))
        sys.stdout.flush()    

In [33]:
import pandas as pd
import math
import pickle  # Para guardar y cargar datos de Python en archivos binarios

save_every = 100000
limit = 5000000
#Guarda el numero de registros procesados
progreso_merged_data = None

# Itera a través de las filas del DataFrame original y crea nuevos registros por género
for index, row in merged_data.iterrows():
    contador(index, 1000)
    if index + 1 >= limit:
        break
    generos = row['genres']
    for genero in generos:
        new_row = row.copy()  # Copia la fila original
        new_row['genres'] = genero  # Asigna el género actual
        new_rows.append(new_row)  # Agrega el nuevo registro a la lista

    
    if (index + 1) % save_every == 0:  # Verifica si es el momento de guardar
        # Crea un nuevo DataFrame con los registros separados por género
        new_df = pd.DataFrame(new_rows)

        # Guarda el DataFrame en un archivo
        filename = f"bloquesData//progreso_iter_{index + 1}.pkl"
        with open(filename, 'wb') as file:
            pickle.dump(new_df, file)

        # Limpia la lista de nuevos registros para la próxima iteración
        new_rows = []


1576000

#### Carga el df de pickle

In [32]:
import pandas as pd
import pickle
import os

#Cargo el df original
#merged_data = pd.read_csv('data//df_recomendacion.csv')
print("Tamaño del df: ",len(merged_data))

# Carga el DataFrame desde el archivo
# Directorio que contiene los archivos Pickle
directorio = "bloquesData"  # Reemplaza con la ruta de tu directorio

# Lista para almacenar los DataFrames
dataframes = []

# Enumera los archivos en el directorio y ordena alfabéticamente
archivos = sorted([archivo for archivo in os.listdir(directorio) if archivo.startswith("progreso_iter_") and archivo.endswith(".pkl")])

# Cargar los DataFrames desde los archivos Pickle
for archivo in archivos:
    ruta_completa = os.path.join(directorio, archivo)
    with open(ruta_completa, "rb") as file:
        df = pickle.load(file)
        dataframes.append(df)

# Concatenar los DataFrames en uno solo
resultado_final = pd.concat(dataframes, ignore_index=True)

print("Tamaño del df parcial: ",len(merged_data))
resultado_final.head(8)

Tamaño del df:  4173927
Tamaño del df parcial:  4173927


,user_id,item_id,playtime_forever,genres,publisher,title,recommend
0,UTNerd24,211360,81,Adventure,Kyle Pulver,Offspring Fling!,NaN
1,UTNerd24,211360,81,Indie,Kyle Pulver,Offspring Fling!,NaN
2,UTNerd24,108800,0,Action,Electronic Arts,Crysis 2 - Maximum Edition,NaN
3,UTNerd24,113200,4,Action,Edmund McMillen,The Binding of Isaac,NaN
4,UTNerd24,113200,4,Adventure,Edmund McMillen,The Binding of Isaac,NaN
5,UTNerd24,113200,4,Indie,Edmund McMillen,The Binding of Isaac,NaN
6,UTNerd24,113200,4,RPG,Edmund McMillen,The Binding of Isaac,NaN
7,UTNerd24,730,261,Action,Valve,Counter-Strike: Global Offensive,NaN


Continua con el loop

In [ ]:
print("Inicio en el registro: ",len(new_df))
print("Tamaño del df: ",len(merged_data))

for index in range(len(new_df), len(merged_data)):
    row = merged_data.iloc[index]
    contador(index, 200)
    generos = row['genres']
    for genero in generos:
        # Crea un nuevo registro utilizando la fila original y modifica solo la columna 'genres'
        new_row = row.copy()
        new_row['genres'] = genero
        new_rows.append(new_row)  # Agrega el nuevo registro a la lista

# Crea un nuevo DataFrame con los registros separados por género
new_df = pd.DataFrame(new_rows)

tam = len(new_df)
print("Tamaño del DF:", tam)
new_df.head(0)
       


In [ ]:
import os
import pandas as pd
import pickle

# Directorio que contiene los archivos Pickle
directorio = "ruta/al/directorio"  # Reemplaza con la ruta de tu directorio

# Lista para almacenar los DataFrames
dataframes = []

# Enumera los archivos en el directorio
for archivo in os.listdir(directorio):
    if archivo.endswith(".pkl"):  # Verifica si el archivo es un archivo Pickle
        ruta_completa = os.path.join(directorio, archivo)
        with open(ruta_completa, "rb") as file:
            df = pickle.load(file)
            dataframes.append(df)

# Concatena los DataFrames en uno solo
resultado_final = pd.concat(dataframes, ignore_index=True)

# Ahora "resultado_final" contiene el DataFrame completo con todos los datos.


In [ ]:
# Guardar el DataFrame en un archivo CSV
total = len(resultado_final)
print("Total de registros: ",total)
resultado_final.to_csv('data//df_recomendacion(full).csv', index=False) 
resultado_final.head(0)

In [ ]:
import math
import pandas as pd

# Inicializa una lista para almacenar los nuevos registros
new_rows = []
limit = 10000000

# Itera a través de las filas del DataFrame original y crea nuevos registros por género
for index, row in merged_data.iterrows():
    contador(index, 100)
    if index + 1 >= limit:  # Limita las iteraciones
        break
    generos = row['genres']
    for genero in generos:
        # Crea un nuevo registro utilizando la fila original y modifica solo la columna 'genres'
        new_row = row.copy()
        new_row['genres'] = genero
        new_rows.append(new_row)  # Agrega el nuevo registro a la lista

# Crea un nuevo DataFrame con los registros separados por género
new_df = pd.DataFrame(new_rows)

tam = len(new_df)
print("Tamaño del DF:", tam)
new_df.head(0)


In [25]:
tam = len(new_df)
print("Tamaño del DF: ",tam)
new_df.head(0)

new_df

,user_id,item_id,playtime_forever,genres,publisher,title,recommend
0,76561197970982479,10,6,['Action'],Valve,Counter-Strike,NaN
1,76561197970982479,20,0,['Action'],Valve,Team Fortress Classic,NaN
2,76561197970982479,30,7,['Action'],Valve,Day of Defeat,NaN
3,76561197970982479,40,0,['Action'],Valve,Deathmatch Classic,NaN
4,76561197970982479,50,0,['Action'],Valve,Half-Life: Opposing Force,NaN
5,76561197970982479,60,0,['Action'],Valve,Ricochet,NaN
6,76561197970982479,70,0,['Action'],Valve,Half-Life,NaN
7,76561197970982479,130,0,['Action'],Valve,Half-Life: Blue Shift,NaN
8,76561197970982479,300,4733,['Action'],Valve,Day of Defeat: Source,NaN
9,76561197970982479,240,1853,['Action'],Valve,Counter-Strike: Source,NaN


In [26]:
from sklearn.metrics.pairwise import cosine_similarity

# Elegir las características para el cálculo de similitud
features = merged_data[["item_id", "genres"]]

# Pivotea los datos para obtener una matriz de géneros por juego
pivot_data = features.pivot_table(index="item_id", columns="genres", aggfunc=lambda x: 1, fill_value=0)

# Calcula la matriz de similitud del coseno
similarities = cosine_similarity(pivot_data)

# Convierte la matriz de similitud en un DataFrame para un fácil acceso
similarity_matrix = pd.DataFrame(similarities, index=pivot_data.index, columns=pivot_data.index)

similarity_matrix.head()

item_id,10,20,30,40,50,60,70,80,130,220,...,526790,527340,527440,527510,527520,527810,527890,527900,528660,530720
item_id,,,,,,,,,,,,,,,,,,,,,
10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
def get_recommendations(input_game, similarity_matrix, game_data, n=10):
    if isinstance(input_game, str):
        # Si se proporciona el nombre del juego de entrada, busca su ID
        input_game_id = game_data[game_data["title"] == input_game]["item_id"].iloc[0]
    else:
        # Si se proporciona el ID del juego de entrada, úsalo directamente
        input_game_id = input_game
    
    similar_scores = similarity_matrix[input_game_id].sort_values(ascending=False)
    recommended_games = similar_scores.drop(input_game_id).head(n)
    
    # Obtiene los IDs de los juegos recomendados
    recommended_game_ids = recommended_games.index
    
    # Obtiene los nombres de los juegos recomendados
    recommended_game_names = game_data.loc[recommended_game_ids, "title"]
    
    # Combina los IDs y nombres de los juegos recomendados en un DataFrame
    recommendations = pd.DataFrame({
        "item_id": recommended_game_ids,
        "title": recommended_game_names
    })
    
    return recommendations

# Ejemplo de recomendación para un juego de entrada (puedes usar nombre o ID)
input_game = "Counter-Strike"  # O input_game = ID_del_juego
recommendations = get_recommendations(input_game, similarity_matrix, merged_data)
print(recommendations)


         item_id                          title
item_id                                        
32680      32680               Joint Task Force
324830    324830                            NaN
32610      32610                            NaN
32620      32620                   PlanetSide 2
32630      32630         One Finger Death Punch
32640      32640     Wolfenstein: The New Order
32650      32650                            NaN
32660      32660                 Reign Of Kings
32670      32670                  Day of Defeat
32770      32770  Blood Bowl® Legendary Edition
